In [1]:
import pickle
import sys
from collections import defaultdict
from pathlib import Path

import cv2
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import parc
import scipy.cluster.hierarchy as sch
import seaborn as sns
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1.inset_locator import mark_inset, zoomed_inset_axes
from skimage import measure, exposure
import skimage.io
from sklearn.preprocessing import MinMaxScaler
from typing import List


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Import path
module_path = str(Path.cwd().parents[0])
if module_path not in sys.path:
    sys.path.append(module_path)

module_path = str(Path.cwd().parents[0] / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
import utils as utils
from config import *

In [5]:
csv_file = data_meta / "info_combined.csv"

# Read dataframe containing images information
df = pd.read_csv(csv_file)

In [6]:
# Parameters
min_cluster_size = 500
min_intensity = 0.3
random_seed = 1

# Read data

In [7]:
def get_location(df, location):
    imgs_path = []
    markers = []

    df = df[(df.Location == location)]
    for row in df.itertuples():
        imgs_path.append(row.Path)
        markers.append(row.Marker)
    return imgs_path, markers

def combined_imgs(imgs: List[str]) -> np.ndarray:
    """
    Return an multiplex image of dimension (num markers, height,width)
    """
    if not imgs:
        raise Exception("You passed a empty list of images path")
    pixels = []
    for img_path in imgs:
        img = skimage.io.imread(img_path)
        p2, p98 = np.percentile(img, (1, 99))
        img = exposure.rescale_intensity(img, in_range=(p2, p98))
        mean = np.mean(img)
        img_rescale = cv2.subtract(img, np.ones(img.shape, dtype=img.dtype)*np.mean(img).astype(img.dtype))
        pixels.append(img_rescale)
    return np.stack(pixels)

def get_multiplex_pix(df, masks, save=False):
    df_appended = []
    for idx, location in enumerate(masks.keys()):
        imgs, markers = get_location(df, location)
        pixels = combined_imgs(imgs)

        mask_cyto = masks[location]["cyto"]
        mask_nuclei = masks[location]["nuclei"]
        cell, _, _ = utils.qc_nuclei(mask_cyto, mask_nuclei)
        labels = [n for n in np.unique(cell) if n > 0]

        # Extracted all multiplex pixels in cell
        rows, cols = np.where(np.isin(cell, labels))
        cell_pixels = pixels[:, rows, cols]

        # Create dataframe
        df_pixels = pd.DataFrame(cell_pixels.T, columns=markers)
        df_pixels["Location"] = location
        df_pixels["X"] = rows
        df_pixels["Y"] = cols
        df_pixels["Id"] = cell[rows, cols]
        df_dapi = df_pixels.filter(like="Hoeschst")
        df_pixels = df_pixels.drop(df_dapi, axis=1)
        df_pixels.insert(
            0, column="Hoeschst", value=df_dapi.mean(axis=1).astype(np.uint16)
        )
        df_appended.append(df_pixels)

    df_pixels = pd.concat(df_appended, ignore_index=True)

    if save:
        df_pixels.to_csv(data_meta / "pixel_intensity.csv", index=False)

    return df_pixels

In [ ]:
df_pixels = pd.read_csv(data_meta / "pixel_intensity.csv")

In [ ]:
df_pixels

In [ ]:
# masks = utils.get_masks(data_mask)
# df_pixels = get_multiplex_pix(df, masks, save=True)

In [ ]:
locations = df_pixels.Location.unique()
location = locations[:9]

# for location in locations:
#     pixels = df_pixels[df_pixels.Location==location].iloc[:, 3:-4]
#     pixels.drop(columns=['Concanavalin A'], inplace=True)
    
#     # Scale data
#     scaler = MinMaxScaler()
#     x_scaled = scaler.fit_transform(pixels)
#     pixels_scaled = pd.DataFrame(x_scaled, columns=pixels.columns)
    
#     # Filter low intensity pixels
#     pixel_dark = pixels_scaled.le(min_intensity).all(axis=1)
#     pixels_bright = pixels_scaled[~pixel_dark]

#     # Parc clustering
#     parc1 = parc.PARC(pixels_bright.values, jac_weighted_edges = False, small_pop = 2000, random_seed=0)
#     parc1.run_PARC() 
#     parc_labels = parc1.labels
    
#     graph = parc1.knngraph_full()
#     embeddings = parc1.run_umap_hnsw(pixels_bright.values, graph, random_state = 0)
    
#     with open(data_meta / f"{location}_{min_ntensity}.pickle", "wb") as f:
#         pickle.dump([parc_labels, embeddings, pixels_bright], f)

In [ ]:
location

In [ ]:
pixels = df_pixels[df_pixels.Location.isin(location)].iloc[:, 3:-4]
pixels.drop(columns=['Concanavalin A'], inplace=True)
display(pixels.max())

# Scale data
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(pixels)
pixels_scaled = pd.DataFrame(x_scaled, columns=pixels.columns)
display(pixels_scaled.describe())

In [ ]:
pixel_dark = pixels_scaled.le(min_intensity).all(axis=1)
display(pixel_dark.value_counts())
pixels_bright = pixels_scaled[~pixel_dark]
display(pixels_bright.head())

# Clustering

In [ ]:
parc1 = parc.PARC(pixels_bright.values, jac_weighted_edges = False, small_pop = 2000, random_seed=0)
parc1.run_PARC() 
parc_labels = parc1.labels

In [ ]:
# graph = parc1.knngraph_full()
# embeddings = parc1.run_umap_hnsw(pixels_bright.values, graph, random_state = 0)

In [ ]:
import scipy

%%time
sparse_mat = scipy.sparse.csr_matrix(pixels_bright.values)
print('Umap embedding')
embeddings = umap.UMAP(metric='cosine', n_neighbors=100).fit_transform(sparse_mat)


In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
X, Y = embeddings[:, 0], embeddings[:, 1]
ax.scatter(X, Y, c=parc_labels, s=2, cmap='jet')
ax.axis("off")

In [ ]:
with open(data_meta / f"1_{min_intensity}.pickle", "wb") as f:
    pickle.dump([parc_labels, embeddings, pixels_bright], f)